In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
from tqdm import tqdm
import cv2
from keras import Sequential
from keras import models
from keras import layers
from keras.layers import *

Using TensorFlow backend.


In [2]:
fog_sub_1_path = "/home/aniketh/Documents/fog-parkinsons/rps/120s/individual_data/fog/fog_sub_1"
fog_sub_2_path = "/home/aniketh/Documents/fog-parkinsons/rps/120s/individual_data/fog/fog_sub_2"
fog_sub_3_path = "/home/aniketh/Documents/fog-parkinsons/rps/120s/individual_data/fog/fog_sub_3"
fog_sub_4_path = "/home/aniketh/Documents/fog-parkinsons/rps/120s/individual_data/fog/fog_sub_4"
fog_sub_5_path = "/home/aniketh/Documents/fog-parkinsons/rps/120s/individual_data/fog/fog_sub_5"
fog_sub_6_path = "/home/aniketh/Documents/fog-parkinsons/rps/120s/individual_data/fog/fog_sub_6"
fog_sub_7_path = "/home/aniketh/Documents/fog-parkinsons/rps/120s/individual_data/fog/fog_sub_7"
fog_sub_8_path = "/home/aniketh/Documents/fog-parkinsons/rps/120s/individual_data/fog/fog_sub_8"
fog_sub_9_path = "/home/aniketh/Documents/fog-parkinsons/rps/120s/individual_data/fog/fog_sub_9"
fog_sub_10_path = "/home/aniketh/Documents/fog-parkinsons/rps/120s/individual_data/fog/fog_sub_10"


non_fog_sub_1_path = "/home/aniketh/Documents/fog-parkinsons/rps/120s/individual_data/non_fog/non_fog_sub_1"
non_fog_sub_2_path = "/home/aniketh/Documents/fog-parkinsons/rps/120s/individual_data/non_fog/non_fog_sub_2"
non_fog_sub_3_path = "/home/aniketh/Documents/fog-parkinsons/rps/120s/individual_data/non_fog/non_fog_sub_3"
non_fog_sub_4_path = "/home/aniketh/Documents/fog-parkinsons/rps/120s/individual_data/non_fog/non_fog_sub_4"
non_fog_sub_5_path = "/home/aniketh/Documents/fog-parkinsons/rps/120s/individual_data/non_fog/non_fog_sub_5"
non_fog_sub_6_path = "/home/aniketh/Documents/fog-parkinsons/rps/120s/individual_data/non_fog/non_fog_sub_6"
non_fog_sub_7_path = "/home/aniketh/Documents/fog-parkinsons/rps/120s/individual_data/non_fog/non_fog_sub_7"
non_fog_sub_8_path = "/home/aniketh/Documents/fog-parkinsons/rps/120s/individual_data/non_fog/non_fog_sub_8"
non_fog_sub_9_path = "/home/aniketh/Documents/fog-parkinsons/rps/120s/individual_data/non_fog/non_fog_sub_9"
non_fog_sub_10_path = "/home/aniketh/Documents/fog-parkinsons/rps/120s/individual_data/non_fog/non_fog_sub_10"

## Patient-wise model using cross validation

## Subject 1

In [4]:

data = []
labels = []
IMG_SIZE = 100


def create_train_data(): 
    

    for img in tqdm(os.listdir(fog_sub_1_path)): 

        path = os.path.join(fog_sub_1_path, img) 

        # loading the image from the path and then converting them into 
        # greyscale for easier covnet prob 
        img = cv2.imread(path) 

        # resizing the image for processing them in the covnet 
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) 

        img = np.array(img)/255.0

        img = img - np.mean(img)

        # final step-forming the training data list with numpy array of the images 
        data.append(img)
        labels.append(1)


    for img in tqdm(os.listdir(non_fog_sub_1_path)): 

        path = os.path.join(non_fog_sub_1_path, img) 

        # loading the image from the path and then converting them into 
        # greyscale for easier covnet prob 
        img = cv2.imread(path) 

        # resizing the image for processing them in the covnet 
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) 
        
        img = np.array(img)/255.0
        
        img = img - np.mean(img)

        # final step-forming the training data list with numpy array of the images 
        data.append(img)
        labels.append(0)


import os
create_train_data()
data_np = np.array(data)
labels_np = np.array(labels)
os.chdir("/home/aniketh/Documents/fog-parkinsons/rps/120s/individual_data")
np.save("data_sub_1.npy", data_np)
np.save("labels_sub_1.npy", labels_np)
    # shuffling of the training data to preserve the random state of our data 
#     os.chdir("/home/aniketh/Documents/fog-parkinsons") 
#     np.save("data_all_120s.npy", data)
#     np.save("labels_all_120s.npy", labels)
def load_model():
    
    IMG_SIZE = 100
    model = Sequential()

    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold

if __name__ == "__main__":
    load_model()
    kFold = StratifiedKFold(n_splits=10, shuffle = True)
    for train, test in kFold.split(data_np,labels_np):
        model = None
        model = load_model()
        train_evaluate(model, data_np[train], labels_np[train], data_np[test], labels_np[test])

100%|██████████| 261/261 [00:57<00:00,  4.34it/s]


Epoch 1/1
53/53 [==============================] - 0s 8ms/step
0.21412595994067643 0.9811320765963141
Epoch 1/1
52/52 [==============================] - 0s 6ms/step
0.02849354181671515 1.0
Epoch 1/1
52/52 [==============================] - 0s 6ms/step
0.0794844740571884 1.0
Epoch 1/1
52/52 [==============================] - 0s 7ms/step
0.03006530922945016 1.0
Epoch 1/1
52/52 [==============================] - 0s 7ms/step
0.058061902751573004 1.0
Epoch 1/1
52/52 [==============================] - 0s 8ms/step
0.06032171007245779 1.0
Epoch 1/1
51/51 [==============================] - 0s 8ms/step
0.07240082170156871 1.0
Epoch 1/1
51/51 [==============================] - 0s 9ms/step
0.2567694675469515 0.9411764740943909
Epoch 1/1
51/51 [==============================] - 0s 10ms/step
0.11166219318322107 1.0
Epoch 1/1
51/51 [==============================] - 1s 10ms/step
0.16328893773549913 1.0


## Subject 2

In [5]:
fog_sub_2_path = "/home/aniketh/Documents/fog-parkinsons/rps/120s/individual_data/fog/fog_sub_2"
non_fog_sub_2_path = "/home/aniketh/Documents/fog-parkinsons/rps/120s/individual_data/non_fog/non_fog_sub_2"

data = []
labels = []
IMG_SIZE = 100


def create_train_data(): 
    

    for img in tqdm(os.listdir(fog_sub_2_path)): 

        path = os.path.join(fog_sub_2_path, img) 

        # loading the image from the path and then converting them into 
        # greyscale for easier covnet prob 
        img = cv2.imread(path) 

        # resizing the image for processing them in the covnet 
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) 

        img = np.array(img)/255.0

        img = img - np.mean(img)

        # final step-forming the training data list with numpy array of the images 
        data.append(img)
        labels.append(1)


    for img in tqdm(os.listdir(non_fog_sub_2_path)): 

        path = os.path.join(non_fog_sub_2_path, img) 

        # loading the image from the path and then converting them into 
        # greyscale for easier covnet prob 
        img = cv2.imread(path) 

        # resizing the image for processing them in the covnet 
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) 
        
        img = np.array(img)/255.0
        
        img = img - np.mean(img)

        # final step-forming the training data list with numpy array of the images 
        data.append(img)
        labels.append(0)

import os
create_train_data()
data_np = np.array(data)
labels_np = np.array(labels)
os.chdir("/home/aniketh/Documents/fog-parkinsons/rps/120s/individual_data")
np.save("data_sub_2.npy", data_np)
np.save("labels_sub_2.npy", labels_np)
# shuffling of the training data to preserve the random state of our data 
#     os.chdir("/home/aniketh/Documents/fog-parkinsons") 
#     np.save("data_all_120s.npy", data)
#     np.save("labels_all_120s.npy", labels)

100%|██████████| 179/179 [00:40<00:00,  3.79it/s]


In [89]:
from sklearn.model_selection import StratifiedKFold

if __name__ == "__main__":
    load_model()
    kFold = StratifiedKFold(n_splits=10, shuffle = True)
    for train, test in kFold.split(data_np,labels_np):
        model = None
        model = load_model()
        train_evaluate(model, data_np[train], labels_np[train], data_np[test], labels_np[test])

Epoch 1/1
36/36 [==============================] - 3s 82ms/step
0.15816586216290793 1.0
Epoch 1/1
36/36 [==============================] - 3s 87ms/step
0.337293666580485 0.861111119389534
Epoch 1/1
36/36 [==============================] - 3s 87ms/step
0.15656787631774527 1.0
Epoch 1/1
36/36 [==============================] - 3s 88ms/step
0.1736670984990067 0.9722222238779068
Epoch 1/1
36/36 [==============================] - 3s 91ms/step
0.4320659504996406 1.0
Epoch 1/1
36/36 [==============================] - 3s 93ms/step
0.15532300536223273 0.9722222238779068
Epoch 1/1
36/36 [==============================] - 4s 98ms/step
0.09960675542242825 0.9722222238779068
Epoch 1/1
36/36 [==============================] - 4s 98ms/step
0.10141421238788301 1.0
Epoch 1/1
36/36 [==============================] - 4s 124ms/step
0.20059242906669775 1.0
Epoch 1/1
35/35 [==============================] - 4s 121ms/step
0.12493988605482238 1.0


## Subject 3

In [7]:

data = []
labels = []
IMG_SIZE = 100


def create_train_data(): 
    

    for img in tqdm(os.listdir(fog_sub_3_path)): 

        path = os.path.join(fog_sub_3_path, img) 

        # loading the image from the path and then converting them into 
        # greyscale for easier covnet prob 
        img = cv2.imread(path) 

        # resizing the image for processing them in the covnet 
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) 

        img = np.array(img)/255.0

        img = img - np.mean(img)

        # final step-forming the training data list with numpy array of the images 
        data.append(img)
        labels.append(1)


    for img in tqdm(os.listdir(non_fog_sub_3_path)): 

        path = os.path.join(non_fog_sub_3_path, img) 

        # loading the image from the path and then converting them into 
        # greyscale for easier covnet prob 
        img = cv2.imread(path) 

        # resizing the image for processing them in the covnet 
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) 
        
        img = np.array(img)/255.0
        
        img = img - np.mean(img)

        # final step-forming the training data list with numpy array of the images 
        data.append(img)
        labels.append(0)

import os
create_train_data()
data_np = np.array(data)
labels_np = np.array(labels)
os.chdir("/home/aniketh/Documents/fog-parkinsons/rps/120s/individual_data")
np.save("data_sub_3.npy", data_np)
np.save("labels_sub_3.npy", labels_np)
# shuffling of the training data to preserve the random state of our data 
#     os.chdir("/home/aniketh/Documents/fog-parkinsons") 
#     np.save("data_all_120s.npy", data)
#     np.save("labels_all_120s.npy", labels)


 55%|█████▍    | 125/228 [00:28<00:22,  4.68it/s]


100%|██████████| 228/228 [00:50<00:00,  4.96it/s]

In [11]:
len(data)

453

In [8]:
from sklearn.model_selection import StratifiedKFold

if __name__ == "__main__":
    load_model()
    kFold = StratifiedKFold(n_splits=5, shuffle = True)
    for train, test in kFold.split(data_np,labels_np):
        model = None
        model = load_model()
        train_evaluate(model, data_np[train], labels_np[train], data_np[test], labels_np[test])

Epoch 1/1
91/91 [==============================] - 1s 8ms/step
0.16776507263900814 0.9780219793319702
Epoch 1/1
91/91 [==============================] - 1s 9ms/step
0.11049575073091866 0.9780219793319702
Epoch 1/1
91/91 [==============================] - 1s 9ms/step
0.14461547076947742 0.9450549483299255
Epoch 1/1
90/90 [==============================] - 1s 9ms/step
0.3010417726796327 0.866666672958268
Epoch 1/1
90/90 [==============================] - 1s 10ms/step
0.10195211409073737 0.9333333373069763


## Subject 4 - no fog

Hence we create a np file of just non_fog events

In [9]:

data = []
labels = []
IMG_SIZE = 100


def create_train_data(): 
    
    for img in tqdm(os.listdir(non_fog_sub_4_path)): 

        path = os.path.join(non_fog_sub_4_path, img) 

        # loading the image from the path and then converting them into 
        # greyscale for easier covnet prob 
        img = cv2.imread(path) 

        # resizing the image for processing them in the covnet 
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) 
        
        img = np.array(img)/255.0
        
        img = img - np.mean(img)

        # final step-forming the training data list with numpy array of the images 
        data.append(img)
        labels.append(0)

import os
create_train_data()
data_np = np.array(data)
labels_np = np.array(labels)
os.chdir("/home/aniketh/Documents/fog-parkinsons/rps/120s/individual_data")
np.save("data_sub_4.npy", data_np)
np.save("labels_sub_4.npy", labels_np)
# shuffling of the training data to preserve the random state of our data 
#     os.chdir("/home/aniketh/Documents/fog-parkinsons") 
#     np.save("data_all_120s.npy", data)
#     np.save("labels_all_120s.npy", labels)



100%|██████████| 302/302 [01:09<00:00,  4.54it/s]

## Subject 5

In [10]:

data = []
labels = []
IMG_SIZE = 100


def create_train_data(): 
    

    for img in tqdm(os.listdir(fog_sub_5_path)): 

        path = os.path.join(fog_sub_5_path, img) 

        # loading the image from the path and then converting them into 
        # greyscale for easier covnet prob 
        img = cv2.imread(path) 

        # resizing the image for processing them in the covnet 
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) 

        img = np.array(img)/255.0

        img = img - np.mean(img)

        # final step-forming the training data list with numpy array of the images 
        data.append(img)
        labels.append(1)


    for img in tqdm(os.listdir(non_fog_sub_5_path)): 

        path = os.path.join(non_fog_sub_5_path, img) 

        # loading the image from the path and then converting them into 
        # greyscale for easier covnet prob 
        img = cv2.imread(path) 

        # resizing the image for processing them in the covnet 
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) 
        
        img = np.array(img)/255.0
        
        img = img - np.mean(img)

        # final step-forming the training data list with numpy array of the images 
        data.append(img)
        labels.append(0)

import os
create_train_data()
data_np = np.array(data)
labels_np = np.array(labels)
os.chdir("/home/aniketh/Documents/fog-parkinsons/rps/120s/individual_data")
np.save("data_sub_5.npy", data_np)
np.save("labels_sub_5.npy", labels_np)
# shuffling of the training data to preserve the random state of our data 
#     os.chdir("/home/aniketh/Documents/fog-parkinsons") 
#     np.save("data_all_120s.npy", data)
#     np.save("labels_all_120s.npy", labels)
from sklearn.model_selection import StratifiedKFold

if __name__ == "__main__":
    load_model()
    kFold = StratifiedKFold(n_splits=5, shuffle = True)
    for train, test in kFold.split(data_np,labels_np):
        model = None
        model = load_model()
        train_evaluate(model, data_np[train], labels_np[train], data_np[test], labels_np[test])


 54%|█████▍    | 128/235 [00:29<00:26,  4.07it/s]


100%|██████████| 235/235 [00:52<00:00,  4.46it/s]

Epoch 1/1
96/96 [==============================] - 1s 10ms/step
0.28086233177843195 0.8958333379899462
Epoch 1/1
96/96 [==============================] - 1s 10ms/step
0.2502740216053401 0.8854166719441613
Epoch 1/1
96/96 [==============================] - 1s 11ms/step
0.215497606148953 0.9062500024835268
Epoch 1/1
95/95 [==============================] - 1s 12ms/step
0.2646842300891876 0.8736842164867803
Epoch 1/1
95/95 [==============================] - 1s 12ms/step
0.22747649124970562 0.894736846810893


## Subject 6

In [11]:
data = []
labels = []
IMG_SIZE = 100


def create_train_data(): 
    

    for img in tqdm(os.listdir(fog_sub_6_path)): 

        path = os.path.join(fog_sub_6_path, img) 

        # loading the image from the path and then converting them into 
        # greyscale for easier covnet prob 
        img = cv2.imread(path) 

        # resizing the image for processing them in the covnet 
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) 

        img = np.array(img)/255.0

        img = img - np.mean(img)

        # final step-forming the training data list with numpy array of the images 
        data.append(img)
        labels.append(1)


    for img in tqdm(os.listdir(non_fog_sub_6_path)): 

        path = os.path.join(non_fog_sub_6_path, img) 

        # loading the image from the path and then converting them into 
        # greyscale for easier covnet prob 
        img = cv2.imread(path) 

        # resizing the image for processing them in the covnet 
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) 
        
        img = np.array(img)/255.0
        
        img = img - np.mean(img)

        # final step-forming the training data list with numpy array of the images 
        data.append(img)
        labels.append(0)

import os
create_train_data()
data_np = np.array(data)
labels_np = np.array(labels)
os.chdir("/home/aniketh/Documents/fog-parkinsons/rps/120s/individual_data")
np.save("data_sub_6.npy", data_np)
np.save("labels_sub_6.npy", labels_np)
# shuffling of the training data to preserve the random state of our data 
#     os.chdir("/home/aniketh/Documents/fog-parkinsons") 
#     np.save("data_all_120s.npy", data)
#     np.save("labels_all_120s.npy", labels)
from sklearn.model_selection import StratifiedKFold

if __name__ == "__main__":
    load_model()
    kFold = StratifiedKFold(n_splits=5, shuffle = True)
    for train, test in kFold.split(data_np,labels_np):
        model = None
        model = load_model()
        train_evaluate(model, data_np[train], labels_np[train], data_np[test], labels_np[test])


 58%|█████▊    | 147/252 [00:33<00:23,  4.41it/s]


100%|██████████| 252/252 [00:58<00:00,  4.55it/s]

Epoch 1/1
100/100 [==============================] - 1s 12ms/step
0.2649648584425449 0.9700000017881394
Epoch 1/1
100/100 [==============================] - 1s 12ms/step
0.022057742276228965 1.0
Epoch 1/1
99/99 [==============================] - 1s 13ms/step
0.02927769725258001 1.0
Epoch 1/1
98/98 [==============================] - 1s 14ms/step
0.05413904536648521 0.9897959189755576
Epoch 1/1
98/98 [==============================] - 1s 15ms/step
0.04963461477879663 1.0


## Subject 7

In [12]:
data = []
labels = []
IMG_SIZE = 100


def create_train_data(): 
    

    for img in tqdm(os.listdir(fog_sub_7_path)): 

        path = os.path.join(fog_sub_7_path, img) 

        # loading the image from the path and then converting them into 
        # greyscale for easier covnet prob 
        img = cv2.imread(path) 

        # resizing the image for processing them in the covnet 
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) 

        img = np.array(img)/255.0

        img = img - np.mean(img)

        # final step-forming the training data list with numpy array of the images 
        data.append(img)
        labels.append(1)


    for img in tqdm(os.listdir(non_fog_sub_7_path)): 

        path = os.path.join(non_fog_sub_7_path, img) 

        # loading the image from the path and then converting them into 
        # greyscale for easier covnet prob 
        img = cv2.imread(path) 

        # resizing the image for processing them in the covnet 
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) 
        
        img = np.array(img)/255.0
        
        img = img - np.mean(img)

        # final step-forming the training data list with numpy array of the images 
        data.append(img)
        labels.append(0)

import os
create_train_data()
data_np = np.array(data)
labels_np = np.array(labels)
os.chdir("/home/aniketh/Documents/fog-parkinsons/rps/120s/individual_data")
np.save("data_sub_7.npy", data_np)
np.save("labels_sub_7.npy", labels_np)
# shuffling of the training data to preserve the random state of our data 
#     os.chdir("/home/aniketh/Documents/fog-parkinsons") 
#     np.save("data_all_120s.npy", data)
#     np.save("labels_all_120s.npy", labels)
from sklearn.model_selection import StratifiedKFold

if __name__ == "__main__":
    load_model()
    kFold = StratifiedKFold(n_splits=5, shuffle = True)
    for train, test in kFold.split(data_np,labels_np):
        model = None
        model = load_model()
        train_evaluate(model, data_np[train], labels_np[train], data_np[test], labels_np[test])


 86%|████████▌ | 193/225 [00:43<00:07,  4.39it/s]


100%|██████████| 225/225 [00:51<00:00,  4.34it/s]

Epoch 1/1
92/92 [==============================] - 2s 16ms/step
0.023542336198379813 1.0
Epoch 1/1
92/92 [==============================] - 2s 16ms/step
0.08675290808668765 0.9782608708609706
Epoch 1/1
92/92 [==============================] - 2s 17ms/step
0.09211670957586687 1.0
Epoch 1/1
92/92 [==============================] - 2s 18ms/step
0.02991743686447001 1.0
Epoch 1/1
91/91 [==============================] - 2s 18ms/step
0.016100814913982874 1.0


## Subject 8

In [14]:
data = []
labels = []
IMG_SIZE = 100


def create_train_data(): 
    
    
    for img in tqdm(os.listdir(fog_sub_8_path)): 

        path = os.path.join(fog_sub_8_path, img) 

        # loading the image from the path and then converting them into 
        # greyscale for easier covnet prob 
        img = cv2.imread(path) 

        # resizing the image for processing them in the covnet 
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) 

        img = np.array(img)/255.0

        img = img - np.mean(img)

        # final step-forming the training data list with numpy array of the images 
        data.append(img)
        labels.append(1)


    for img in tqdm(os.listdir(non_fog_sub_8_path)): 

        path = os.path.join(non_fog_sub_8_path, img) 

        # loading the image from the path and then converting them into 
        # greyscale for easier covnet prob 
        img = cv2.imread(path) 

        # resizing the image for processing them in the covnet 
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) 

        img = np.array(img)/255.0

        img = img - np.mean(img)

        # final step-forming the training data list with numpy array of the images 
        data.append(img)
        labels.append(0)

import os
create_train_data()
data_np = np.array(data)
labels_np = np.array(labels)
os.chdir("/home/aniketh/Documents/fog-parkinsons/rps/120s/individual_data")
np.save("data_sub_8.npy", data_np)
np.save("labels_sub_8.npy", labels_np)
# shuffling of the training data to preserve the random state of our data 
#     os.chdir("/home/aniketh/Documents/fog-parkinsons") 
#     np.save("data_all_120s.npy", data)
#     np.save("labels_all_120s.npy", labels)
from sklearn.model_selection import StratifiedKFold

if __name__ == "__main__":
    load_model()
    kFold = StratifiedKFold(n_splits=5, shuffle = True)
    for train, test in kFold.split(data_np,labels_np):
        model = None
        model = load_model()
        train_evaluate(model, data_np[train], labels_np[train], data_np[test], labels_np[test])


100%|██████████| 90/90 [00:17<00:00,  4.93it/s]

Epoch 1/1
40/40 [==============================] - 2s 41ms/step
0.5682739093899727 0.6000000014901161
Epoch 1/1
40/40 [==============================] - 2s 39ms/step
0.6171398162841797 0.5500000007450581
Epoch 1/1
40/40 [==============================] - 2s 43ms/step
0.4185718521475792 0.9250000007450581
Epoch 1/1
39/39 [==============================] - 2s 45ms/step
0.5985360168493711 0.5641025648667262
Epoch 1/1
39/39 [==============================] - 2s 47ms/step
0.6289137846384293 0.5384615411361059


## Subject 9

In [17]:
data = []
labels = []
IMG_SIZE = 100


def create_train_data(): 
    
    for img in tqdm(os.listdir(fog_sub_9_path)): 

        path = os.path.join(fog_sub_9_path, img) 

        # loading the image from the path and then converting them into 
        # greyscale for easier covnet prob 
        img = cv2.imread(path) 

        # resizing the image for processing them in the covnet 
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) 

        img = np.array(img)/255.0

        img = img - np.mean(img)

        # final step-forming the training data list with numpy array of the images 
        data.append(img)
        labels.append(1)


    for img in tqdm(os.listdir(non_fog_sub_9_path)): 

        path = os.path.join(non_fog_sub_9_path, img) 

        # loading the image from the path and then converting them into 
        # greyscale for easier covnet prob 
        img = cv2.imread(path) 

        # resizing the image for processing them in the covnet 
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) 
        
        img = np.array(img)/255.0
        
        img = img - np.mean(img)

        # final step-forming the training data list with numpy array of the images 
        data.append(img)
        labels.append(0)

import os
create_train_data()
data_np = np.array(data)
labels_np = np.array(labels)
os.chdir("/home/aniketh/Documents/fog-parkinsons/rps/120s/individual_data")
np.save("data_sub_9.npy", data_np)
np.save("labels_sub_9.npy", labels_np)
# shuffling of the training data to preserve the random state of our data 
#     os.chdir("/home/aniketh/Documents/fog-parkinsons") 
#     np.save("data_all_120s.npy", data)
#     np.save("labels_all_120s.npy", labels)
from sklearn.model_selection import StratifiedKFold

if __name__ == "__main__":
    load_model()
    kFold = StratifiedKFold(n_splits=5, shuffle = True)
    for train, test in kFold.split(data_np,labels_np):
        model = None
        model = load_model()
        train_evaluate(model, data_np[train], labels_np[train], data_np[test], labels_np[test])


 62%|██████▏   | 126/202 [00:29<00:20,  3.77it/s]


100%|██████████| 202/202 [00:46<00:00,  4.31it/s]

Epoch 1/1
87/87 [==============================] - 2s 24ms/step
0.2598015695981596 0.9195402329680563
Epoch 1/1
87/87 [==============================] - 2s 25ms/step
0.3103422435193226 0.8390804652510018
Epoch 1/1
86/86 [==============================] - 2s 26ms/step
0.2399695954870346 0.9767441874326661
Epoch 1/1
86/86 [==============================] - 2s 28ms/step
0.28815522911243663 0.8837209371633308
Epoch 1/1
86/86 [==============================] - 2s 28ms/step
0.27503698951629707 0.8837209338712138


## subject 10 - no fog

hence we create np files of just non_fog events

In [18]:

data = []
labels = []
IMG_SIZE = 100


def create_train_data(): 
    
    

    for img in tqdm(os.listdir(non_fog_sub_10_path)): 

        path = os.path.join(non_fog_sub_10_path, img) 

        # loading the image from the path and then converting them into 
        # greyscale for easier covnet prob 
        img = cv2.imread(path) 

        # resizing the image for processing them in the covnet 
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) 
        
        img = np.array(img)/255.0
        
        img = img - np.mean(img)

        # final step-forming the training data list with numpy array of the images 
        data.append(img)
        labels.append(0)

import os
create_train_data()
data_np = np.array(data)
labels_np = np.array(labels)
os.chdir("/home/aniketh/Documents/fog-parkinsons/rps/120s/individual_data")
np.save("data_sub_10.npy", data_np)
np.save("labels_sub_10.npy", labels_np)




100%|██████████| 263/263 [01:04<00:00,  4.23it/s]